# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [15]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [16]:
# topDirName = 'Training'
topDirName = '5_11_2019_MM_DNN'

Lump together all subdirectories with the same classification label. 

In [3]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [4]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [5]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Use a CNN to transform image sequences to a pickle file and organize the results.

In [6]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

Use a pretrained CNN model (here using DenseNet121 because it has worked well in the past) with the final classifier layer chopped off.

In [7]:
# Import the CNN model and move it to CUDA if available.
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [8]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = torch.FloatTensor(len(files),1,1024)
            for i in range(len(files)):
                FinalOutput[i] = output[i]
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [9]:
transform_to_pickle(model,topDirName)

Transforming image stack 5_11_2019_MM_DNN/Absent/XS_greenBox_Negative_11-Nov-2019_16-13-07
0.014812620352540364% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_20-40-09
0.029625240705080727% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_Negative_13-Nov-2019_13-07-52
0.0444378610576211% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_15-37-14
0.059250481410161454% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Spaced_Negative_06-Nov-2019_12-01-55
0.07406310176270181% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_Negative_13-Nov-2019_16-51-59
0.0888757221152422% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/XS_greenBox_Negative_11-Nov-2019_15-39-43
0.10368834246778254% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_filledTube_Negative_05-Nov-2019_14-20-48
0.11850096282032291% complete.
Trans

0.9776329432676641% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_20-34-05
0.9924455636202044% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_20-53-20
1.0072581839727448% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_label_Negative_18-Nov-2019_16-39-43
1.022070804325285% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_filledTube_Negative_05-Nov-2019_14-30-07
1.0368834246778253% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_redBox_label_Negative_20-Nov-2019_09-19-05
1.0516960450303658% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_11-32-10
1.0665086653829063% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_11-12-35
1.0813212857354466% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Spaced_Negative_06-Nov-2019

1.9552658865353283% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-37-23
1.9700785068878686% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_14-Nov-2019_15-23-35
1.9848911272404088% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_10-37-11
1.9997037475929493% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_15-38-36
2.0145163679454896% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_label_Negative_18-Nov-2019_17-53-08
2.02932898829803% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Angle_Negative_06-Nov-2019_12-15-01
2.04414160865057% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/XS_greenBox_Negative_11-Nov-2019_16-32-26
2.0589542290031106% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_Negative_13-Nov-2019_13-41-49
2.07

2.932898829802992% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_redBox_label_Negative_20-Nov-2019_09-22-31
2.9477114501555324% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Angle_Negative_06-Nov-2019_14-11-26
2.962524070508073% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_filledBottle_H_Negative_30-Oct-2019_13-34-54
2.9773366908606134% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/XS_filledTube_Negative_05-Nov-2019_16-43-07
2.992149311213154% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_greenBox_Negative_13-Nov-2019_15-48-55
3.006961931565694% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_18-Nov-2019_14-02-22
3.0217745519182344% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_11-16-56
3.036587172270775% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_14-Nov-2019_15-03-01
3.

3.9105317730706566% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_filledBottle_H_Negative_30-Oct-2019_15-45-18
3.925344393423197% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_filledTube_Negative_05-Nov-2019_15-26-12
3.940157013775737% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/6S_redBox_label_Negative_20-Nov-2019_09-38-42
3.9549696341282776% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_19-42-04
3.9697822544808177% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-36-39
3.984594874833358% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_14-Nov-2019_13-36-09
3.9994074951858987% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/XS_filledTube_Negative_05-Nov-2019_16-59-53
4.014220115538439% complete.
Transforming image stack 5_11_2019_MM_DNN/Absent/7_whiteBox_Panpass_Negative_15-Nov-2019_18-50-04

...

94.87483335802104% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_18-43-40
94.88964597837357% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_19-57-24
94.90445859872611% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_filledTube_05-Nov-2019_11-00-01
94.91927121907867% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_redBox_label_19-Nov-2019_14-17-38
94.9340838394312% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_17-13-10
94.94889645978374% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBottle_H_29-Oct-2019_10-36-54
94.96370908013627% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_17-37-53
94.97852170048881% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBox_label_19-Nov-2019_16-55-13
94.99333432084136% complete.
Transforming image stack 5_11_2019_MM_DN

95.9265294030514% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_filledBottle_V_30-Oct-2019_11-43-20
95.94134202340395% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_20-23-15
95.95615464375649% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_filledTube_05-Nov-2019_10-12-47
95.97096726410902% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_18-12-21
95.98577988446156% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_13-Nov-2019_02-11-12
96.0005925048141% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBottle_V_30-Oct-2019_08-55-17
96.01540512516664% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_20-11-35
96.03021774551918% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_greenBox_label_19-Nov-2019_11-02-09
96.04503036587172% complete.
Transforming image stack 5_11_2019_MM_D

96.97822544808177% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_12-36-16
96.9930380684343% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBox_label_19-Nov-2019_15-04-16
97.00785068878685% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_19-28-06
97.02266330913939% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBottle_V_30-Oct-2019_09-14-24
97.03747592949192% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBottle_H_29-Oct-2019_10-17-30
97.05228854984446% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_20-13-31
97.067101170197% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_redBox_label_19-Nov-2019_16-36-25
97.08191379054955% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_filledTube_05-Nov-2019_10-59-14
97.0967264109021% complete.
Transforming image stack 5_11_2019_MM_DNN/Prese

98.02992149311213% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_13-Nov-2019_07-55-59
98.04473411346467% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_11-42-08
98.05954673381721% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_19-47-40
98.07435935416974% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_19-13-08
98.08917197452229% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_redBox_label_19-Nov-2019_14-15-08
98.10398459487485% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_13-Nov-2019_02-10-22
98.11879721522737% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_filledBottle_V_30-Oct-2019_11-06-58
98.13360983557992% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_fiiledTube_05-Nov-2019_12-36-25
98.14842245593245% complete.
Transforming image stack 5_11_2019_MM_D

99.0816175381425% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_fiiledTube_05-Nov-2019_12-54-14
99.09643015849504% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_greenBox_label_19-Nov-2019_10-44-55
99.11124277884757% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_20-14-43
99.12605539920011% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_17-22-06
99.14086801955267% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_whiteBox_label_12-Nov-2019_18-50-58
99.1556806399052% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/7_fiiledTube_05-Nov-2019_12-33-16
99.17049326025774% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/6S_filledBottle_H_29-Oct-2019_15-56-12
99.18530588061027% complete.
Transforming image stack 5_11_2019_MM_DNN/Present/XS_whiteBox_label_11-Nov-2019_19-01-41
99.20011850096282% complete.
Transforming image stack 5_11_2019_MM_DNN/

Clean up the directory, removing everything but the pickle files.

In [22]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [23]:
import random

In [24]:
# finalDirName = 'Split_Data'
finalDirName = 'Split_Data_5_11_2019_MM_DNN'

In [28]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        #Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [29]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)